In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.5. Group Distribution Analysis/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.5. Group Distribution Analysis/Output')
cur_date = "042222"

library(readxl)
library(tidyverse)


#reading in file
proteomics_df = data.frame(read_excel("041822_Imputed_Proteomics_Data.xlsx"))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
head(proteomics_df)

,ID,Set,Treatment,Protein,Value
,<chr>,<chr>,<chr>,<chr>,<dbl>
1,CoCulture_Exposed1,Set1,CoCulture_Exposed,P19012,476617.6
2,CoCulture_Exposed1,Set1,CoCulture_Exposed,P19013,1077007.5
3,CoCulture_Exposed1,Set1,CoCulture_Exposed,O43897,2245997.6
4,CoCulture_Exposed1,Set1,CoCulture_Exposed,P20700,2388716.6
5,CoCulture_Exposed1,Set1,CoCulture_Exposed,P22102,1790626.6
6,CoCulture_Exposed1,Set1,CoCulture_Exposed,O75367,874143.2


In [3]:
proteomics_df = proteomics_df %>%
    #just want a column that specifies the treatment and exposure separately
    separate(Treatment, c("Treatment", "Exposure"), sep = "_") 

head(proteomics_df)

,ID,Set,Treatment,Exposure,Protein,Value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,CoCulture_Exposed1,Set1,CoCulture,Exposed,P19012,476617.6
2,CoCulture_Exposed1,Set1,CoCulture,Exposed,P19013,1077007.5
3,CoCulture_Exposed1,Set1,CoCulture,Exposed,O43897,2245997.6
4,CoCulture_Exposed1,Set1,CoCulture,Exposed,P20700,2388716.6
5,CoCulture_Exposed1,Set1,CoCulture,Exposed,P22102,1790626.6
6,CoCulture_Exposed1,Set1,CoCulture,Exposed,O75367,874143.2


In [4]:
#explain why I split this
proteomics_df_split = proteomics_df %>%
    group_by(Set) %>%
    group_split()

set1_proteomics_df = proteomics_df_split[[1]]
set2_proteomics_df = proteomics_df_split[[2]]

head(set1_proteomics_df)

ID,Set,Treatment,Exposure,Protein,Value
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
CoCulture_Exposed1,Set1,CoCulture,Exposed,P19012,476617.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,P19013,1077007.5
CoCulture_Exposed1,Set1,CoCulture,Exposed,O43897,2245997.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,P20700,2388716.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,P22102,1790626.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,O75367,874143.2


Running t tests to determine if there are statistically significant distribution differences in co-culture unexposed vs. exposed and tri-culture unexposed vs. exposed. 

In [5]:
t_test_values = function(df, set_id){
    #"""
    #Running t tests after filtering for set, treatment, exposure, and protein using a loop. 
    #Ultimately using this test to compare proteins (unexposed vs. exposed).

    #:param: subsetted dataframe, empty dataframe
    #:output: a dataframe containing the set, treatment, exposure, protein, u stat, p value, p adj

    #"""
    treatments = unique(df$Treatment)
    proteins = unique(df$Protein)
    
    values_df = data.frame()
    #iterating through each tx, protein

    for(i in 1:length(treatments)){
        for(j in 1:length(proteins)){
            #unexposed df
            unexposed_df = df %>%
                filter(Treatment == treatments[i], Exposure == "Vehicle", Protein == proteins[j])
            #exposed df
            exposed_df = df %>%
                filter(Treatment == treatments[i], Exposure == "Exposed", Protein == proteins[j])

            #t test
            t_test = t.test(unexposed_df$Value, exposed_df$Value)
            
            #calculating FC to get directionality
            FC = log2(mean(unexposed_df$Value)/mean(exposed_df$Value))

            #contains smoking status compared, compartment, cytokine, u stat, and p value
            values_vector = cbind(set_id, treatments[i], proteins[j], FC, t_test$statistic, t_test$p.value)
            values_df = rbind(values_df, values_vector)
            }
        }

    
    #adding col names
    colnames(values_df) = c("Set", "Treatment", "Protein", "log2 FC", "Statistic", "P Value")
    
    
    #calculating padj values
    padj_vector = c()
    #adjusting within each treatment
    for (i in 1:length(treatments)){
        `P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")
        padj_vector = c(padj_vector, `P Adj`)
    }
    
    values_df = cbind(values_df, padj_vector) %>%
        rename(`P Adj` = padj_vector)
    
    return(values_df)
}

In [6]:
# calling fn
set1_t_test = t_test_values(set1_proteomics_df, "Set1")
set2_t_test = t_test_values(set2_proteomics_df, "Set2")

In [7]:
# creating 1 df
t_test_df = rbind(set1_t_test, set2_t_test)
head(t_test_df)

,Set,Treatment,Protein,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
t,Set1,CoCulture,P19012,0.60331489149206,0.567224863429138,0.8207093
t1,Set1,CoCulture,P19013,-0.139436949314188,0.893842444121871,0.9672176
t2,Set1,CoCulture,O43897,-1.28817058222099,0.241597197697623,0.7149305
t3,Set1,CoCulture,P20700,-0.401101776101729,0.698456072213535,0.8649296
t4,Set1,CoCulture,P22102,1.09076873867442,0.311878044109208,0.7329387
t5,Set1,CoCulture,O75367,-0.603646331227886,0.563303637544858,0.8186874


In [3]:
#calculating overall avg. abundance and within smoking groups
avg_abundance_df = proteomics_df %>%
    group_by(Set, Protein) %>%
    summarize(Avg_Abundance = mean(Value))

#stratified by treatment
avg_abundance_df_stratified = proteomics_df %>%
    group_by(Set, Treatment, Protein) %>%
    summarize(Avg_Abundance_stratified = mean(Value))

head(avg_abundance_df)

`summarise()` has grouped output by 'Set'. You can override using the `.groups`
argument.
`summarise()` has grouped output by 'Set', 'Treatment'. You can override using
the `.groups` argument.


Set,Protein,Avg_Abundance
<chr>,<chr>,<dbl>
Set1,A0A096LP49,51897126
Set1,A0A2R8Y4Y8,1745284
Set1,A1L4H1,19473534
Set1,A2I7N0,62531054
Set1,A2I7N1,12761148
Set1,A2I7N3,71043165


Set,Treatment,Protein,Avg_Abundance_stratified
<chr>,<chr>,<chr>,<dbl>
Set1,CoCulture_Exposed,A0A096LP49,50430413
Set1,CoCulture_Exposed,A0A2R8Y4Y8,1547459
Set1,CoCulture_Exposed,A1L4H1,14958036
Set1,CoCulture_Exposed,A2I7N0,69348055
Set1,CoCulture_Exposed,A2I7N1,14022002
Set1,CoCulture_Exposed,A2I7N3,76870390


In [10]:
#changing the stratified dfs to making combining easier
#first splitting dfs
split_avg_abundance_stratified = avg_abundance_df_stratified %>%
    group_by(Treatment) %>%
    group_split()

#adding treatment to the title
coculture_exposed_avg_abundance = split_avg_abundance_stratified[[1]] %>%
    select(-Treatment) %>%
    rename(Coculture_Exposed_Avg_Abundance = Avg_Abundance_stratified)
coculture_vehicle_avg_abundance = split_avg_abundance_stratified[[2]] %>%
    select(-Treatment) %>%
    rename(Coculture_Vehicle_Avg_Abundance = Avg_Abundance_stratified)
triculture_exposed_avg_abundance = split_avg_abundance_stratified[[1]] %>%
    select(-Treatment) %>%
    rename(Triculture_Exposed_Avg_Abundance = Avg_Abundance_stratified)
triculture_vehicle_avg_abundance = split_avg_abundance_stratified[[2]] %>%
    select(-Treatment) %>%
    rename(Triculture_Vehicle_Avg_Abundance = Avg_Abundance_stratified)

head(coculture_exposed_avg_abundance)

Set,Protein,Coculture_Exposed_Avg_Abundance
<chr>,<chr>,<dbl>
Set1,A0A096LP49,50430413
Set1,A0A2R8Y4Y8,1547459
Set1,A1L4H1,14958036
Set1,A2I7N0,69348055
Set1,A2I7N1,14022002
Set1,A2I7N3,76870390


In [13]:
# combining into 1 df
stratified_abundance_df = inner_join(inner_join(coculture_exposed_avg_abundance, coculture_vehicle_avg_abundance), 
      inner_join(triculture_exposed_avg_abundance, triculture_vehicle_avg_abundance))
abundance_df = inner_join(avg_abundance_df, stratified_abundance_df)
head(abundance_df)

Joining, by = c("Set", "Protein")
Joining, by = c("Set", "Protein")
Joining, by = c("Set", "Protein")
Joining, by = c("Set", "Protein")


Set,Protein,Avg_Abundance,Coculture_Exposed_Avg_Abundance,Coculture_Vehicle_Avg_Abundance,Triculture_Exposed_Avg_Abundance,Triculture_Vehicle_Avg_Abundance
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Set1,A0A096LP49,51897126,50430413,53363840,50430413,53363840
Set1,A0A2R8Y4Y8,1745284,1547459,1943110,1547459,1943110
Set1,A1L4H1,19473534,14958036,23989033,14958036,23989033
Set1,A2I7N0,62531054,69348055,55714053,69348055,55714053
Set1,A2I7N1,12761148,14022002,11500294,14022002,11500294
Set1,A2I7N3,71043165,76870390,65215940,76870390,65215940


In [8]:
# exporting
write.csv(t_test_df, paste0(Output,"/", cur_date, "_Distribution_Analysis.csv"), row.names = FALSE)